In [1]:
# Install Kaggle API
!pip install kaggle

# Create directory
!mkdir -p ~/.kaggle

# Copy kaggle.json (must be uploaded first)
!cp /content/kaggle.json ~/.kaggle/

# Change permissions
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API Successfully Configured!")

Kaggle API Successfully Configured!


In [2]:
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics

Cloning into 'ultralytics'...
remote: Enumerating objects: 76162, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 76162 (delta 80), reused 94 (delta 54), pack-reused 75995 (from 2)
Receiving objects: 100% (76162/76162), 40.60 MiB | 16.24 MiB/s, done.
Resolving deltas: 100% (57224/57224), done.
/content/ultralytics


In [3]:
!pip install -e .
import torch
print("GPU:", torch.cuda.get_device_name(0))

Obtaining file:///content/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.233-0.editable-py3-none-any.whl size=23167 sha256=c1404c61fcb73c1ff4254662fa29d58944517a309f6a9249917e1cbd1fc8e9f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-qrck7le_/wheels/60/e0/59/e2f034f296abbdca5c21e3f5be76b9ca685f13c7bd17f8b58c
Successfully built ultralytics
GPU: Tesla T4


In [4]:
import os, shutil

print("🧹 Cleaning datasets directory...")
if os.path.exists('/content/datasets'):
    shutil.rmtree('/content/datasets')

os.makedirs('/content/datasets')

%cd /content/datasets

🧹 Cleaning datasets directory...
/content/datasets


In [5]:
print("⬇️ Downloading Brain Tumor Dataset...")
!kaggle datasets download -d pkdarabi/medical-image-dataset-brain-tumor-detection

⬇️ Downloading Brain Tumor Dataset...
Dataset URL: https://www.kaggle.com/datasets/pkdarabi/medical-image-dataset-brain-tumor-detection
License(s): Attribution 4.0 International (CC BY 4.0)
 51% 150M/297M [00:00<00:00, 1.56GB/s]
100% 297M/297M [00:07<00:00, 44.3MB/s]


In [6]:
print("📦 Unzipping...")
!unzip -q medical-image-dataset-brain-tumor-detection.zip
!rm medical-image-dataset-brain-tumor-detection.zip

📦 Unzipping...


In [7]:
import os

print("🔍 Locating data.yaml...")

yaml_path = None

for root, dirs, files in os.walk('/content/datasets'):
    if 'data.yaml' in files:
        yaml_path = os.path.join(root, 'data.yaml')
        break

if yaml_path is None:
    raise FileNotFoundError("❌ data.yaml not found!")

dataset_root = os.path.abspath(os.path.dirname(yaml_path))

yaml_content = f"""
path: {dataset_root}
train: train/images
val: valid/images
test: test/images

nc: 3
names: ['glioma', 'meningioma', 'pituitary']
"""

with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("✅ FIXED data.yaml at:", yaml_path)
print("📍 Dataset root:", dataset_root)

🔍 Locating data.yaml...
✅ FIXED data.yaml at: /content/datasets/BrainTumor/BrainTumorYolov11/data.yaml
📍 Dataset root: /content/datasets/BrainTumor/BrainTumorYolov11


In [8]:
import re

tasks_path = "/content/ultralytics/ultralytics/nn/tasks.py"

residual_attention_code = """
# --- ResidualAttention START ---
# Residual Attention: A Simple but Effective Method for Multi-Label Recognition
# Trunk branch (identity features) + Mask branch (attention mask)
# Output = trunk * (1 + mask)

class ResidualAttention(nn.Module):
    def __init__(self, c1, c2=None, reduction=16):
        super().__init__()
        if c2 is None:
            c2 = c1

        mid = max(8, c1 // reduction)

        # Mask branch (attention branch)
        self.mask = nn.Sequential(
            nn.Conv2d(c1, mid, kernel_size=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid, c1, kernel_size=1, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        mask = self.mask(x)
        return x * (1 + mask)   # residual attention
# --- ResidualAttention END ---
"""

print("🔧 Injecting ResidualAttention into tasks.py...")

with open(tasks_path, "r") as f:
    lines = f.readlines()

# Check if already present
if any("class ResidualAttention" in line for line in lines):
    print("ℹ️ ResidualAttention already exists — skipping insertion.")
else:
    insert_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith("class BaseModel"):
            insert_index = i
            break

    if insert_index is None:
        print("❌ ERROR: BaseModel not found.")
    else:
        new_lines = lines[:insert_index] + [residual_attention_code + "\n"] + lines[insert_index:]
        with open(tasks_path, "w") as f:
            f.writelines(new_lines)
        print("✅ ResidualAttention inserted above BaseModel.")

        # Register module
        with open(tasks_path, "r") as f:
            content = f.read()

        if "ResidualAttention" not in content:
            content = content.replace("if m in (", "if m in (ResidualAttention, ", 1)
            with open(tasks_path, "w") as f:
                f.write(content)
            print("✅ ResidualAttention registered.")
        else:
            print("ℹ️ Already registered.")

🔧 Injecting ResidualAttention into tasks.py...
✅ ResidualAttention inserted above BaseModel.
ℹ️ Already registered.


In [9]:
residual_yaml = """
# YOLOv8n + Residual Attention

nc: 3  # adjust for his dataset if needed

backbone:
  - [-1, 1, Conv, [64, 3, 2]]
  - [-1, 1, Conv, [128, 3, 2]]
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]

  # Residual Attention at deep feature map
  - [-1, 1, ResidualAttention, [1024]]

head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]
  - [-1, 3, C2f, [256]]

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 13], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 10], 1, Concat, [1]]
  - [-1, 3, C2f, [1024]]

  - [[16, 19, 22], 1, Detect, [nc]]
"""

with open("/content/ultralytics/yolov8n-residual.yaml", "w") as f:
    f.write(residual_yaml)

print("✅ Created yolov8n-residual.yaml")

✅ Created yolov8n-residual.yaml


In [11]:
%cd /content/ultralytics
import os
os.environ["WANDB_DISABLED"] = "true"

!yolo detect train \
data=/content/datasets/BrainTumor/BrainTumorYolov11/data.yaml \
model=/content/ultralytics/yolov8n-residual.yaml \
epochs=50 \
imgsz=640 \
batch=16 \
name=YOLOv8_ResidualAttention

/content/ultralytics
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/BrainTumor/BrainTumorYolov11/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, h